### Import bibliotek 

In [1]:
# %pip install selenium webdriver-manager beautifulsoup4 requests


In [2]:
import os
import json
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException, StaleElementReferenceException
import time

### Definicja narzędzi pomocniczych

In [8]:
def extract(ancestor, selector=None, attribute=None, many=False):
    if selector:
        if many:
            if attribute:
                return [item[attribute].strip() for item in ancestor.select(selector)]
            return [item.text.strip() for item in ancestor.select(selector)]
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None
    if attribute:
        try:
            return ancestor[attribute]
        except TypeError:
            return None
    return ancestor.text.strip()

In [9]:
opinion_schema = {
    "stars": ("span.a-icon-alt",),
    "content": ("span.review-text-content",)
}

### Logowanie do Amazona

In [ ]:
product_id = input("Podaj kod produktu: ")

In [ ]:
with open("./credentials.json", "r") as jf:
    credentials = json.load(jf)
ACCOUNT = credentials["ACCOUNT"]
PASSWORD = credentials["PASSWORD"]

In [ ]:
next_page = f"https://www.amazon.com/product-reviews/{product_id}"
path_to_chromedriver = 'C:\\Users\\kaska\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe'
s = Service(path_to_chromedriver)
driver = webdriver.Chrome(service=s)
driver.get(next_page)
driver.maximize_window()
WebDriverWait(driver, 20).until(EC.element_to_be_clickable(driver.find_element("name", "email"))).send_keys(ACCOUNT)
driver.find_element(by='xpath', value='//*[@id="continue"]/span/input').click()
WebDriverWait(driver, 20).until(EC.element_to_be_clickable(driver.find_element("name", "password"))).send_keys(PASSWORD)
driver.find_element(by='xpath', value='//*[@id="signInSubmit"]').click()


### Pobranie z Amazon.com opinii o wskazanym produkcie

In [ ]:
all_opinions = []
while True:
    opinions = driver.find_elements(by=By.CSS_SELECTOR, value="li.review.aok-relative")
    print(len(opinions))
    for opinion in opinions:
        opinion_html = opinion.get_attribute('innerHTML')
        opinion_dom = BeautifulSoup(opinion_html, 'html.parser')
        single_opinion = {
            key: extract(opinion_dom, *value)
            for key, value in opinion_schema.items()
        }
        all_opinions.append(single_opinion)
    try:
        time.sleep(3)
        driver.find_element(by=By.CSS_SELECTOR, value='li.a-last a').click()
    except NoSuchElementException:
        next_page = None
        break
    

In [ ]:
next_page = f"https://www.ceneo.pl/{product_id}#tab=spec"
response = requests.get(next_page, headers=headers)
if response.status_code == 200:
        page_dom = BeautifulSoup(response.text, "html.parser")
        brand = extract(page_dom, "table.product-spec__group__attributes > tbody > tr:nth-child(2) > td.product-spec__group__attributes__row__value")
        model = extract(page_dom, "table.product-spec__group__attributes > tbody > tr:nth-child(3) > td.product-spec__group__attributes__row__value")

In [ ]:
print(f"Zbieranie opinii dla produktu: {brand} {model}")

### Zapisanie pobranych opinii do plików tekstowych

In [ ]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")
if not os.path.exists("./opinions/en"):
    os.mkdir("./opinions/en")

In [ ]:
for idx, single_opinion in enumerate(all_opinions):
    rating = int(round(float(single_opinion["stars"].split("/")[0].replace(",", "."))/5,3)*100)
    sentiment = "P" if rating >= 50 else "N"
    filename = f"./opinions/pl/{brand.replace(' ', '')}_{model.replace(' ', '')}_{rating:03}_{sentiment}_{idx:03}.txt"
    with open(filename, "w", encoding="utf-8") as tf:
        tf.write(single_opinion["content"])